In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv


In [53]:
load_dotenv()

True

In [ ]:
import os
os.environ.pop("GROQ_API_KEY", None)
print("GROQ:", os.getenv("GROQ_API_KEY"))

GROQ: None


In [20]:
os.environ.pop("GROQ_API_KEY", None)

In [41]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="text-generation"
)


In [42]:
llm = ChatHuggingFace(llm=llm)

In [43]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [44]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = llm.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [45]:
# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [51]:
# Execute
initial_state = {
    'question': 'Who won 2024 T20 WC and by how many runs? Who was Player of match? And give summary about final!!'
}
final_state = workflow.invoke(intial_state)

print(final_state['answer'])

**2024 ICC Men’s T20 World Cup – Final**

| Item | Detail |
|------|--------|
| **Winner** | **India** |
| **Runner‑up** | South Africa |
| **Date & Venue** | 29 June 2024 – Kensington Oval, Bridgetown, Barbados (USA/West Indies co‑hosted tournament) |
| **Result** | India won **by 7 wickets** (with 5 balls to spare) |
| **Player of the Match** | **Jasprit Bumrah** (India) – 3 wickets for 19 runs in 4 overs |

---

### How the final unfolded – a brief summary

1. **South Africa win the toss and elect to bat**  
   * Openers Quinton de Kock (31) and Aiden Markram (38) gave a solid start.  
   * After a quick 70‑run partnership, de Kock was dismissed, and South Africa slumped to 100/5.  
   * A late surge from David Miller (33*) and a quick cameo by Kieron Pollard (18 off 9) took the total to **176/7** in the full 20 overs.

2. **India’s chase**  
   * Openers Ruturaj Gaikwad (73 off 44) and Shubman Gill (31 off 20) gave India a flying start, reaching 100/0 in just 7.2 overs.  
   * Gaik